<a href="https://colab.research.google.com/github/kaushanr/python3-docs/blob/main/Section_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decorators

In [11]:
# Higher order functions 

  # functions calling other functions internally and returning results

def square(num):
  return num**2

def cube(num):
  return num**3

def sum(val,func):
  total = 0
  for num in range(val):
     total += func(num)
  return total

print(sum(7,square))
print(sum(7,cube))

print()

  # functions returning functions 

from random import choice

def make_laugh_func():
  def get_laugh():
    return choice(('HAHAHA','lol','tehehe'))
  return get_laugh

laugh = make_laugh_func()
print(laugh()) # returns the internal function to a variable called 'laugh' which then can be called as a function 
               # to expose the result of the internal function

print()

  # functions with arguments passed in to them

from random import choice

def make_laugh_at_func(person):
  def get_laugh():
    laugh = choice(('HAHAHA','lol','tehehe'))
    return f'{laugh} {person}' # the 'person' argument is passed in to the outer function only but is available for access internally
  return get_laugh

  # closure functions 

      #  A closure is a nested function which has access to a free variable from an enclosing function that has finished its execution.
      # characteristics of a python closure
          # it is a nested function
          # it has access to a free variable in outer scope
          # it is returned from the enclosing function

        # A free variable is a variable that is not bound in the local scope. 
        # In order for closures to work with immutable variables such as numbers and strings, we have to use the nonlocal keyword.
        # immutable - cannot be changed after creation

laugh_at = make_laugh_at_func('Linda')
print(laugh_at())
print(laugh_at())
print(laugh_at())
print(laugh_at())

91
441

lol

tehehe Linda
tehehe Linda
HAHAHA Linda
tehehe Linda


In [5]:
# Decorators

  # decorators are functions 
  # decorators wrap other functions and enhance their behaviour
  # decorators are examples of higher order functions 
  # they have their own syntax - '@' - syntactic sugar

# Decorators as functions 

def be_polite(fn): # takes another external function as an input argument
  def wrapper(): # runs another internal function
    print('What a pleasure to meet you!')
    fn() # executes the called function as well
    print('Have a great day!')
  return wrapper

def greet():
  print('My name is Colt')


greeting = be_polite(greet)
print(greeting())

print()

def be_polite(fn): # takes another external function as an input argument
  def wrapper(): # runs another internal function
    print('What a pleasure to meet you!')
    fn() # executes the called function as well
    print('Have a great day!')
  return wrapper

@be_polite # the 'be_polite' function is being called as a decorative wrapper function on this function
def greet(): # this is the decorated function - not the decorative function above...
  print('My name is Colt')

@be_polite 
def rage():
  print('I HATE YOU!')

print(greet()) # same as calling for 'be_polite(greet)'
print(rage())

What a pleasure to meet you!
My name is Colt
Have a great day!
None

What a pleasure to meet you!
My name is Colt
Have a great day!
None
What a pleasure to meet you!
I HATE YOU!
Have a great day!
None


In [19]:
def shout(fn): # the decorator function
  def wrapper(*args,**kwargs):
    return fn(*args,**kwargs).upper()
  return wrapper

@shout
def greet(name): # the decorated function
  return f'Hi, I\'m {name}.'

@shout
def order(main,side): # use *args/**kwargs to handle more than 1 argument entry in the 'wrapper()' function
  return f'Hi, I\'d like the {main} with a side of {side}, please'

@shout
def lol():
  return 'lol'

print(greet('todd'))
print(order('burger','fries'))
print(lol())

HI, I'M TODD.
HI, I'D LIKE THE BURGER WITH A SIDE OF FRIES, PLEASE
LOL


In [12]:
# Preserving metadata

def log_function_data(fn):
  def wrapper(*args,**kwargs):
    '''Hi, I am a Wrapper Function'''
    print(f'you are about to call : {fn.__name__}')
    print(f'here\'s the documentation : {fn.__doc__}')
    return fn(*args,**kwargs)
  return wrapper


@log_function_data
def add(x,y):
  '''Adds two numbers together''' # docstrings included here
  return x+y

print(add(10,30))
print(add.__doc__) # the original metadata of the decorated function is lost by the wrapper function when called externally
print(add.__name__) # returns only the wrapper function info
help(add) # returns the wrapper functions documentation

print()

  # preserving this data by using the module functools.wraps

from functools import wraps
def log_function_data(fn):
  @wraps(fn) # decorating our wrapping function with 'wraps'
  def wrapper(*args,**kwargs):
    '''Hi, I am a Wrapper Function'''
    print(f'you are about to call : {fn.__name__}')
    print(f'here\'s the documentation : {fn.__doc__}')
    return fn(*args,**kwargs)
  return wrapper


@log_function_data
def add(x,y):
  '''Adds two numbers together''' # docstrings included here
  return x+y

print(add.__doc__) # returns the desired docstring and name of the decorated function
print(add.__name__)
help(add)

you are about to call : add
here's the documentation : Adds two numbers together
40
Hi, I am a Wrapper Function
wrapper
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    Hi, I am a Wrapper Function


Adds two numbers together
add
Help on function add in module __main__:

add(x, y)
    Adds two numbers together



In [10]:
# a model use case of a decorator function

from time import time
from functools import wraps

def speed_test(fn):
  @wraps(fn)
  def wrapper(*args,**kwargs):
    start_time = time()
    result = fn(*args,**kwargs)
    end_time = time()
    print(f'Executing : {fn.__name__}')
    print(f'Elapsed time : {end_time - start_time}')
    return result
  return wrapper


@speed_test
def sum_nums_gen():
  '''sums the numbers in range given by a generator expression'''
  return sum(num for num in range(50000000))

print(sum_nums_gen())
print(sum_nums_gen.__doc__)
print(sum_nums_gen.__name__)

print()

@speed_test
def sum_nums_list():
  '''sums the numbers in range given in a list object'''
  return sum([num for num in range(50000000)])

print(sum_nums_list())
print(sum_nums_list.__doc__)

Executing : sum_nums_gen
Elapsed time : 3.2081778049468994
1249999975000000
sums the numbers in range given by a generator expression
sum_nums_gen

Executing : sum_nums_list
Elapsed time : 4.738628625869751
1249999975000000
sums the numbers in range given in a list object


In [12]:
# Coding exercise

'''
@show_args
def do_nothing(*args, **kwargs):
    pass

do_nothing(1, 2, 3,a="hi",b="bye")

# Should print (on two lines):
# Here are the args: (1, 2, 3)
# Here are the kwargs: {"a": "hi", "b": "bye"}
'''

from functools import wraps

def show_args(fn):
  @wraps(fn)
  def wrapper(*args,**kwargs):
    print(f'Here are the args: {args}')
    print(f'Here are the kwargs: {kwargs}')
    return fn(*args,**kwargs)
  return wrapper

@show_args
def do_nothing(*args, **kwargs):
    pass

do_nothing(1, 2, 3,a="hi",b="bye")

Here are the args: (1, 2, 3)
Here are the kwargs: {'a': 'hi', 'b': 'bye'}


In [18]:
# Passing argument restrictions with decorator functions 

from functools import wraps 

def ensure_no_kwargs(fn):
  @wraps(fn)
  def wrapper(*args,**kwargs):
    if kwargs:
      raise ValueError('No **kwargs allowed please!')
    return fn(*args,**kwargs)
  return wrapper

@ensure_no_kwargs
def greet(name):
  return f'hi there {name}!'


print(greet('Matilde'))
print(greet(name = 'Colt'))

hi there Matilde!


ValueError: ignored

In [22]:
# Coding exercise

'''
@double_return 
def add(x, y):
    return x + y
    
add(1, 2) # [3, 3]

@double_return
def greet(name):
    return "Hi, I'm " + name

greet("Colt") # ["Hi, I'm Colt", "Hi, I'm Colt"]
'''
from functools import wraps

def double_return(fn):
  @wraps(fn)
  def wrapper(*args,**kwargs):
    return [fn(*args,**kwargs)] * 2
  return wrapper

@double_return 
def add(x, y):
    return x + y
    
print(add(1, 2)) # [3, 3]

@double_return
def greet(name):
    return "Hi, I'm " + name

print(greet("Colt")) # ["Hi, I'm Colt", "Hi, I'm Colt"]

[3, 3]
["Hi, I'm Colt", "Hi, I'm Colt"]


In [10]:
# Coding exercise

'''
@ensure_fewer_than_three_args
def add_all(*nums):
    return sum(nums)

add_all() # 0
add_all(1) # 1
add_all(1,2) # 3
add_all(1,2,3) # "Too many arguments!"
add_all(1,2,3,4,5,6) # "Too many arguments!"
'''

from functools import wraps

def ensure_fewer_than_three_args(fn):
  @wraps(fn)
  def wrapper(*args,**kwargs):
    if len(args) > 2:
      return 'Too many arguments!'
    print(args)
    return fn(*args,**kwargs)
  return wrapper

@ensure_fewer_than_three_args
def add_all(*args): # the args tuple is not unpacked here, the entire tuple is passed into the function and summed
    return sum(args)

print(add_all())
print(add_all(1))
print(add_all(1,2))
print(add_all(1,2,3))

  # Note* - for *args unpacking the def some_func(a,b,c) - some_func(*nums) - nums = [1,3,5]



()
0
(1,)
1
(1, 2)
3
Too many arguments!


In [21]:
# Coding exercise

'''
@only_ints 
def add(x, y):
    return x + y
    
add(1, 2) # 3
add("1", "2") # "Please only invoke with integers."
'''

from functools import wraps

def only_ints(fn):
  @wraps(fn)
  def wrapper(*args):
    while all(map(lambda item: type(item) == int,args)):
      return fn(*args)
    return 'Please only invoke with integers.'
  return wrapper

@only_ints 
def add(x, y):
    return x + y

print(add(1, 2))
print(add("1", "2"))

print()

# another solution

def only_ints(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        if any([arg for arg in args if type(arg) != int]): # if any non-int value is present - list will return a Truthy result
            return "Please only invoke with integers."
        return fn(*args, **kwargs)
    return inner

print(add(1, 2))
print(add("1", "2"))

3
Please only invoke with integers.

3
Please only invoke with integers.


In [28]:
# Coding exercise

'''
@ensure_authorized
def show_secrets(*args, **kwargs):
    return "Shh! Don't tell anybody!"

show_secrets(role="admin") # "Shh! Don't tell anybody!"
show_secrets(role="nobody") # "Unauthorized"
show_secrets(a="b") # "Unauthorized"
'''

from functools import wraps

def ensure_authorized(fn):
  @wraps(fn)
  def wrapper(*args,**kwargs):
    if 'role' in kwargs and kwargs['role'] == 'admin':
      return fn(*args,**kwargs)
    return 'Unauthorized'
  return wrapper

@ensure_authorized
def show_secrets(*args, **kwargs):
    return "Shh! Don't tell anybody!"

print(show_secrets(role="admin"))
print(show_secrets(role="nobody"))
print(show_secrets(a="b"))

print()

# alternative solution

def ensure_authorized(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if kwargs.get("role",False) == "admin": # dict method .get() - returns the value of key - if no key returns None by default
            return fn(*args, **kwargs)
        return "Unauthorized"
    return wrapper

print(show_secrets(role="admin"))
print(show_secrets(role="nobody"))
print(show_secrets(a="b"))


Shh! Don't tell anybody!
Unauthorized
Unauthorized


In [41]:
def ensure_authorized(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if kwargs.get("role",False) == "admin": # dict method .get() - returns the value of key - if no key returns None by default - here False
            return fn(*args, **kwargs)
        return "Unauthorized"
    return wrapper

@ensure_authorized
def show_secrets(*args, **kwargs):
    return "Shh! Don't tell anybody!"

print(show_secrets(role="admin"))
print(show_secrets(role="nobody"))
print(show_secrets(a="b"))
help(dict.get)

Shh! Don't tell anybody!
Unauthorized
Unauthorized
Help on method_descriptor:

get(self, key, default=None, /)
    Return the value for key if key is in the dictionary, else default.



In [49]:
# Decorators taking in arguments

  # when we write:
  ## @decorator
    # def func(*args,**kwargs):
    #   pass
  # we're really doing:
    # func = decorator(func) # the decorated function is passed into the decorator function

  # when we write:
    ## @decorator_with_args(arg)
     # def func(*args,**kwargs):
     #   pass
  # we're really doing:
    # func = decorator_with_args(arg)(func) # the argument is first passed into the decorator 
                                            # followed by the decorated function which receives this argument

from functools import wraps

def ensure_first_arg_is(val):
  def inner_func(fn):
    @wraps(fn)
    def wrapper(*args,**kwargs):
      if args and args[0] != val:
        return f'The first arg needs to be {val}'
      return fn(*args,**kwargs)
    return wrapper
  return inner_func


@ensure_first_arg_is('burrito')
def fav_foods(*args):
  return args

print(fav_foods('burrito','ice cream','crepes'))
print(fav_foods('hummus','ice cream','crepes'))

print()

@ensure_first_arg_is(10)
def add_to_ten(num1,num2):
  return num1 + num2

print(add_to_ten(10,23))
print(add_to_ten(11,23))

('burrito', 'ice cream', 'crepes')
The first arg needs to be burrito

33
The first arg needs to be 10


In [62]:
# Enforcing argument types with a decorator

from functools import wraps

def enforce(*types):
  def inner_func(fn):
    @wraps(fn)
    def wrapper(*args,**kwargs):
      newargs = []
      for a,t in zip(args,types):
        print(a,t)
        newargs.append(t(a))
        print(newargs)
      return fn(*newargs,**kwargs) # in this case the decorated function does not *args as an input, 
    return wrapper                 # hence the resulting list needs to be unpacked and fed into the decorated functions arguments
  return inner_func


@enforce(str,int)
def repeat_msg(msg,times):
  for val in range(times):
    print(msg)
    
print(repeat_msg('Hello','3'))

print()

@enforce(float,float)
def divide(a,b):
  return a/b

print(divide('3','5.6'))

Hello <class 'str'>
['Hello']
3 <class 'int'>
['Hello', 3]
Hello
Hello
Hello
None

3 <class 'float'>
[3.0]
5.6 <class 'float'>
[3.0, 5.6]
0.5357142857142857


In [64]:
# Coding exercise

'''
@delay(3)
def say_hi():
    return "hi"

say_hi()
# should print the message "Waiting 3s before running say_hi" to the console
# should then wait 3 seconds
# finally, should invoke say_hi and return "hi"
'''

from functools import wraps
from time import sleep

def delay(val):
  def inner_func(fn):
    @wraps(fn)
    def wrapper(*args,**kwargs):
      print(f'Waiting {val}s before running {fn.__name__}')
      sleep(val)
      return fn(*args,**kwargs)
    return wrapper
  return inner_func

@delay(5)
def say_hi():
    return "hi"

print(say_hi())

Waiting 5s before running say_hi
hi
